# Permutation Importance on Model M040
- Per Type, find features which do not help the model

In [2]:
import pandas as pd
import catboost
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
import numpy as np
import gc

In [3]:
def permutation_importance(model, X_val, y_val, metric, threshold=0.005,
                           minimize=True, verbose=True):
    """
    Reference: https://www.kaggle.com/speedwagon/permutation-importance
    
    Modified by Rob
    """
    results = {}
    
    y_pred = model.predict(X_val)
    
    results['base_score'] = metric(y_val, y_pred)
    if verbose:
        print(f'Base score {results["base_score"]:.5}')

    
    for col in tqdm(X_val.columns):
        freezed_col = X_val[col].copy()

        X_val[col] = np.random.permutation(X_val[col])
        preds = model.predict(X_val, thread_count=20)
        results[col] = metric(y_val, preds)

        X_val[col] = freezed_col
        
        if verbose:
            print(f'column: {col} - {results[col]:.5}')
    
    if minimize:
        bad_features = [k for k in results if results[k] < results['base_score'] + threshold]
    else:
        bad_features = [k for k in results if results[k] > results['base_score'] + threshold]
    bad_features.remove('base_score')
    
    return results, bad_features

In [12]:
types = ['3JHN', '2JHC', '3JHC', '2JHH', '2JHN', '1JHN', '3JHH', '1JHC']
results_dict = {}
bad_features_dict = {}
for t in types:
    train = pd.read_parquet(f'../data/FE016/FE016-train-{t}.parquet')
    x = pd.read_parquet('../data/FE008_train.parquet') # only loading for skeleton not features
    model = catboost.CatBoostRegressor()
    model.load_model('../models/M040/M040-0708_1639-{}-{}.model'.format(t, 1))
    mol_group = x[['molecule_name','type']].copy()
    N_FOLDS = 2
    folds = GroupKFold(n_splits=N_FOLDS)
    X = train[model.feature_names_]
    y = train['scalar_coupling_constant']
    mol_group_type = mol_group.loc[mol_group['type'] == t]['molecule_name']
    for fold_n, (train_idx, valid_idx) in enumerate(folds.split(X, groups=mol_group_type)):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        X_valid = X_valid.copy()
        results, bad_features = permutation_importance(model, X_valid, y_valid, metric=mean_absolute_error)
        results_dict[f] = results
        bad_features_dict[f] = bad_features
        break # Only run for the first fold
    print(f'Bad features for {t}:')
    print(bad_features)
    del train, x, X, y, mol_group_type
    gc.collect()

  0%|          | 0/214 [00:00<?, ?it/s]

Base score 0.11625


  0%|          | 1/214 [00:08<30:08,  8.49s/it]

column: atom1_valence - 0.13909


  1%|          | 2/214 [00:17<30:37,  8.67s/it]

...

100%|█████████▉| 213/214 [22:47<00:04,  4.37s/it]

column: angle_0_10th1_1 - 0.11644


100%|██████████| 214/214 [22:51<00:00,  4.37s/it]

column: angle_0_11th1_1 - 0.1163


NameError: name 'f' is not defined

In [13]:
results_dict[t] = results
bad_features_dict[t] = bad_features

In [14]:
print(f'Bad features for {t}:')
print(bad_features)

Bad features for 3JHN:
['distance_6th_closest_to_0', 'distance_7th_closest_to_0', 'distance_8th_closest_to_0', 'distance_9th_closest_to_0', 'distance_10th_closest_to_0', 'distance_8th_closest_to_1', 'distance_9th_closest_to_1', 'distance_10th_closest_to_1', 'closest_to_0_atomic_mass', 'closest_to_0_spin_multiplicity', '2nd_closest_to_0_atomic_mass', '2nd_closest_to_0_spin_multiplicity', '3rd_closest_to_0_atomic_mass', '3rd_closest_to_0_spin_multiplicity', '4th_closest_to_0_atomic_mass', '4th_closest_to_0_spin_multiplicity', '5th_closest_to_0_atomic_mass', '5th_closest_to_0_valence', '5th_closest_to_0_spin_multiplicity', '6th_closest_to_0_atomic_mass', '6th_closest_to_0_valence', '6th_closest_to_0_spin_multiplicity', '7th_closest_to_0_atomic_mass', '7th_closest_to_0_valence', '7th_closest_to_0_spin_multiplicity', '8th_closest_to_0_atomic_mass', '8th_closest_to_0_valence', '8th_closest_to_0_spin_multiplicity', '9th_closest_to_0_atomic_mass', '9th_closest_to_0_valence', '9th_closest_to_0_

In [15]:
results

{'base_score': 0.11624966229611856,
 'atom1_valence': 0.13908507750474156,
 'distance': 0.30409684102762097,
 'distance_closest_to_0': 0.1314930921257668,
 'distance_2nd_closest_to_0': 0.2035061385958581,
 'distance_3rd_closest_to_0': 0.13593251497072856,
 'distance_4th_closest_to_0': 0.13592902237180302,
 'distance_5th_closest_to_0': 0.12418844171562442,
 'distance_6th_closest_to_0': 0.12048359490026912,
 'distance_7th_closest_to_0': 0.1197150686224358,
 'distance_8th_closest_to_0': 0.11867837399979411,
 'distance_9th_closest_to_0': 0.11976977457551821,
 'distance_10th_closest_to_0': 0.11788356951574526,
 'distance_closest_to_1': 0.12886166641921612,
 'distance_2nd_closest_to_1': 0.1268584643965523,
 'distance_3rd_closest_to_1': 0.2013288641931263,
 'distance_4th_closest_to_1': 0.1276491289447391,
 'distance_5th_closest_to_1': 0.137498661570282,
 'distance_6th_closest_to_1': 0.12151785667805233,
 'distance_7th_closest_to_1': 0.12805490765777391,
 'distance_8th_closest_to_1': 0.1209469

In [ ]:
types = ['2JHC', '3JHC', '2JHH', '2JHN', '1JHN', '3JHH', '1JHC']
# results_dict = {}
# bad_features_dict = {}
for t in types:
    train = pd.read_parquet(f'../data/FE016/FE016-train-{t}.parquet')
    x = pd.read_parquet('../data/FE008_train.parquet') # only loading for skeleton not features
    model = catboost.CatBoostRegressor()
    model.load_model('../models/M040/M040-0708_1639-{}-{}.model'.format(t, 1))
    mol_group = x[['molecule_name','type']].copy()
    N_FOLDS = 2
    folds = GroupKFold(n_splits=N_FOLDS)
    X = train[model.feature_names_]
    y = train['scalar_coupling_constant']
    mol_group_type = mol_group.loc[mol_group['type'] == t]['molecule_name']
    for fold_n, (train_idx, valid_idx) in enumerate(folds.split(X, groups=mol_group_type)):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        X_valid = X_valid.copy()
        results, bad_features = permutation_importance(model, X_valid, y_valid, metric=mean_absolute_error)
        results_dict[t] = results
        bad_features_dict[t] = bad_features
        break # Only run for the first fold
    print(f'Bad features for {t}:')
    print(bad_features)
    del train, x, X, y, mol_group_type
    gc.collect()

  0%|          | 0/220 [00:00<?, ?it/s]

Base score 0.2185


  0%|          | 1/220 [01:21<4:56:49, 81.32s/it]

column: atom1_valence - 0.72811


  1%|          | 2/220 [02:43<4:56:14, 81.53s/it]

...

column: 10th_closest_to_0_atomic_mass - 0.21942


 24%|██▍       | 53/220 [1:11:12<3:46:02, 81.21s/it]

column: 10th_closest_to_0_valence - 0.22007


 25%|██▍       | 54/220 [1:12:34<3:45:32, 81.52s/it]

column: 10th_closest_to_0_spin_multiplicity - 0.21869
